In [7]:
import pymongo
import pandas as pd
import re

In [2]:
user = 'team04'
pw = '1111'
host = 'ec2-54-95-8-243.ap-northeast-1.compute.amazonaws.com'
client = pymongo.MongoClient(f'mongodb://{user}:{pw}@{host}:27017/')
db = client.resell

In [36]:
df = pd.DataFrame(db.kream_detail.find())
df

,_id,brand,prod_kr_name,rescent_price,model_no,release_date,colors,release_price,instant_buy_price,instant_sell_price,img_url
0,Jordan 1 Low Vintage Grey,Jordan,조던 1 로우 빈티지 그레이,"200,000",553558-053,-,GREY/WHITE/CREAM,"129,000원","202,000","201,000",https://kream-phinf.pstatic.net/MjAyMjA2MjdfMT...
1,Jordan 1 x Travis Scott x Fragment Retro Low O...,Jordan,조던 1 x 트래비스 스캇 x 프라그먼트 레트로 로우 OG SP 밀리터리 블루,"1,555,000",DM7866-140,21/08/13,SAIL/BLACK/MILITARY BLUE,"189,000원","1,540,000","2,230,000",https://kream-phinf.pstatic.net/MjAyMTA3MzBfNT...
2,Jordan 1 Retro High OG Black Mocha,Jordan,조던 1 레트로 하이 OG 블랙 모카,"702,000",555088-105,20/11/13,SAIL/DARK MOCHA-BLACK-BLACK,"199,000원","600,000","818,000",https://kream-phinf.pstatic.net/MjAyMDEwMjJfOC...
3,Jordan 4 Retro White and Black,Jordan,조던 4 레트로 화이트 앤 블랙,"312,000",DH6927-111,22/05/23,WHITE/BLACK/NEUTRAL GREY,"249,000원","299,000","313,000",https://kream-phinf.pstatic.net/MjAyMjA1MDlfOD...
4,Jordan 1 Retro High OG Bleached Coral,Jordan,조던 1 레트로 하이 OG 블리치드 코랄,"244,000",555088-108,22/05/14,WHITE/BLACK/GREY FOG/BLEACHED CORAL,"209,000원","207,000","270,000",https://kream-phinf.pstatic.net/MjAyMjA1MDlfMT...
...,...,...,...,...,...,...,...,...,...,...,...
2999,Jordan 1 Retro High OG SP Gina (Special Box),Jordan,조던 1 레트로 하이 OG SP 지나 (스페셜 박스),-,CD7071-001,19/05/25,BLACK/BLACK-WHITE-VARSITY RED,"$180 (약 234,600원)",-,-,https://kream-phinf.pstatic.net/MjAyMTAyMjJfMS...
3000,Jordan 13 Retro Low Quai 54,Jordan,조던 13 레트로 로우 콰이 54,-,810551-050,15/06/14,BLACK/DARK GREY-WHITE-CHROME,"$250 (약 325,200원)",-,-,https://kream-phinf.pstatic.net/MjAyMDEwMjJfMj...
3001,(GS) Jordan 11 Retro Heiress Velvet,Jordan,(GS) 조던 11 레트로 헤어리스 벨벳,"290,000",852625-650,16/12/18,NIGHT MAROON/METALLIC GOLD-NIGHT MAROON,"$220 (약 286,200원)","340,000","180,000",https://kream-phinf.pstatic.net/MjAyMDEwMjJfND...
3002,Jordan 1 Retro High Flyknit Derek Jeter RE2PECT,Jordan,조던 1 레트로 하이 플라이니트 데릭 지터 RE2PECT,"350,000",AH7233-105,18/02/24,SAIL/METALLIC GOLD-MIDNIGHT NAVY,"$180 (약 234,100원)","380,000","150,000",https://kream-phinf.pstatic.net/MjAyMDEwMjJfNT...


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3004 entries, 0 to 3003
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   _id                 3004 non-null   object
 1   brand               3004 non-null   object
 2   prod_kr_name        3004 non-null   object
 3   rescent_price       3004 non-null   object
 4   model_no            3004 non-null   object
 5   release_date        3004 non-null   object
 6   colors              3004 non-null   object
 7   release_price       3004 non-null   object
 8   instant_buy_price   3004 non-null   object
 9   instant_sell_price  3004 non-null   object
 10  img_url             3004 non-null   object
dtypes: object(11)
memory usage: 258.3+ KB


In [74]:
def price_preprocessing(data):
    data = data.replace('-', '0')
    data = data.replace(',', '')
    
    return int(data)

In [67]:
def date_preprocessing(data):
    data = data.strip()
    if data != '-':
        if int(data[:2]) < 50:
            data = '20' + data
        elif int(data[:2]) >= 50:
            data = '19' + data
    
    data = data.replace('-', '1900/01/01')

    return data

In [52]:
def release_price_preprocessing(data):
    filtered = re.findall('\d[0-9,]*원', data)
    if filtered:
        data = filtered[0].replace('원', '').replace(',', '')
    else:
        data = '0'
    
    return int(data)

In [75]:
df2 = df.copy()
df2

,_id,brand,prod_kr_name,rescent_price,model_no,release_date,colors,release_price,instant_buy_price,instant_sell_price,img_url
0,Jordan 1 Low Vintage Grey,Jordan,조던 1 로우 빈티지 그레이,"200,000",553558-053,-,GREY/WHITE/CREAM,"129,000원","202,000","201,000",https://kream-phinf.pstatic.net/MjAyMjA2MjdfMT...
1,Jordan 1 x Travis Scott x Fragment Retro Low O...,Jordan,조던 1 x 트래비스 스캇 x 프라그먼트 레트로 로우 OG SP 밀리터리 블루,"1,555,000",DM7866-140,21/08/13,SAIL/BLACK/MILITARY BLUE,"189,000원","1,540,000","2,230,000",https://kream-phinf.pstatic.net/MjAyMTA3MzBfNT...
2,Jordan 1 Retro High OG Black Mocha,Jordan,조던 1 레트로 하이 OG 블랙 모카,"702,000",555088-105,20/11/13,SAIL/DARK MOCHA-BLACK-BLACK,"199,000원","600,000","818,000",https://kream-phinf.pstatic.net/MjAyMDEwMjJfOC...
3,Jordan 4 Retro White and Black,Jordan,조던 4 레트로 화이트 앤 블랙,"312,000",DH6927-111,22/05/23,WHITE/BLACK/NEUTRAL GREY,"249,000원","299,000","313,000",https://kream-phinf.pstatic.net/MjAyMjA1MDlfOD...
4,Jordan 1 Retro High OG Bleached Coral,Jordan,조던 1 레트로 하이 OG 블리치드 코랄,"244,000",555088-108,22/05/14,WHITE/BLACK/GREY FOG/BLEACHED CORAL,"209,000원","207,000","270,000",https://kream-phinf.pstatic.net/MjAyMjA1MDlfMT...
...,...,...,...,...,...,...,...,...,...,...,...
2999,Jordan 1 Retro High OG SP Gina (Special Box),Jordan,조던 1 레트로 하이 OG SP 지나 (스페셜 박스),-,CD7071-001,19/05/25,BLACK/BLACK-WHITE-VARSITY RED,"$180 (약 234,600원)",-,-,https://kream-phinf.pstatic.net/MjAyMTAyMjJfMS...
3000,Jordan 13 Retro Low Quai 54,Jordan,조던 13 레트로 로우 콰이 54,-,810551-050,15/06/14,BLACK/DARK GREY-WHITE-CHROME,"$250 (약 325,200원)",-,-,https://kream-phinf.pstatic.net/MjAyMDEwMjJfMj...
3001,(GS) Jordan 11 Retro Heiress Velvet,Jordan,(GS) 조던 11 레트로 헤어리스 벨벳,"290,000",852625-650,16/12/18,NIGHT MAROON/METALLIC GOLD-NIGHT MAROON,"$220 (약 286,200원)","340,000","180,000",https://kream-phinf.pstatic.net/MjAyMDEwMjJfND...
3002,Jordan 1 Retro High Flyknit Derek Jeter RE2PECT,Jordan,조던 1 레트로 하이 플라이니트 데릭 지터 RE2PECT,"350,000",AH7233-105,18/02/24,SAIL/METALLIC GOLD-MIDNIGHT NAVY,"$180 (약 234,100원)","380,000","150,000",https://kream-phinf.pstatic.net/MjAyMDEwMjJfNT...


In [76]:
df2['rescent_price'] = df2['rescent_price'].apply(price_preprocessing)
df2['instant_buy_price'] = df2['instant_buy_price'].apply(price_preprocessing)
df2['instant_sell_price'] = df2['instant_sell_price'].apply(price_preprocessing)
df2['release_date'] = df2['release_date'].apply(date_preprocessing)
df2['release_price'] = df2['release_price'].apply(release_price_preprocessing)

In [77]:
df2

,_id,brand,prod_kr_name,rescent_price,model_no,release_date,colors,release_price,instant_buy_price,instant_sell_price,img_url
0,Jordan 1 Low Vintage Grey,Jordan,조던 1 로우 빈티지 그레이,200000,553558-053,1900/01/01,GREY/WHITE/CREAM,129000,202000,201000,https://kream-phinf.pstatic.net/MjAyMjA2MjdfMT...
1,Jordan 1 x Travis Scott x Fragment Retro Low O...,Jordan,조던 1 x 트래비스 스캇 x 프라그먼트 레트로 로우 OG SP 밀리터리 블루,1555000,DM7866-140,2021/08/13,SAIL/BLACK/MILITARY BLUE,189000,1540000,2230000,https://kream-phinf.pstatic.net/MjAyMTA3MzBfNT...
2,Jordan 1 Retro High OG Black Mocha,Jordan,조던 1 레트로 하이 OG 블랙 모카,702000,555088-105,2020/11/13,SAIL/DARK MOCHA-BLACK-BLACK,199000,600000,818000,https://kream-phinf.pstatic.net/MjAyMDEwMjJfOC...
3,Jordan 4 Retro White and Black,Jordan,조던 4 레트로 화이트 앤 블랙,312000,DH6927-111,2022/05/23,WHITE/BLACK/NEUTRAL GREY,249000,299000,313000,https://kream-phinf.pstatic.net/MjAyMjA1MDlfOD...
4,Jordan 1 Retro High OG Bleached Coral,Jordan,조던 1 레트로 하이 OG 블리치드 코랄,244000,555088-108,2022/05/14,WHITE/BLACK/GREY FOG/BLEACHED CORAL,209000,207000,270000,https://kream-phinf.pstatic.net/MjAyMjA1MDlfMT...
...,...,...,...,...,...,...,...,...,...,...,...
2999,Jordan 1 Retro High OG SP Gina (Special Box),Jordan,조던 1 레트로 하이 OG SP 지나 (스페셜 박스),0,CD7071-001,2019/05/25,BLACK/BLACK-WHITE-VARSITY RED,234600,0,0,https://kream-phinf.pstatic.net/MjAyMTAyMjJfMS...
3000,Jordan 13 Retro Low Quai 54,Jordan,조던 13 레트로 로우 콰이 54,0,810551-050,2015/06/14,BLACK/DARK GREY-WHITE-CHROME,325200,0,0,https://kream-phinf.pstatic.net/MjAyMDEwMjJfMj...
3001,(GS) Jordan 11 Retro Heiress Velvet,Jordan,(GS) 조던 11 레트로 헤어리스 벨벳,290000,852625-650,2016/12/18,NIGHT MAROON/METALLIC GOLD-NIGHT MAROON,286200,340000,180000,https://kream-phinf.pstatic.net/MjAyMDEwMjJfND...
3002,Jordan 1 Retro High Flyknit Derek Jeter RE2PECT,Jordan,조던 1 레트로 하이 플라이니트 데릭 지터 RE2PECT,350000,AH7233-105,2018/02/24,SAIL/METALLIC GOLD-MIDNIGHT NAVY,234100,380000,150000,https://kream-phinf.pstatic.net/MjAyMDEwMjJfNT...


In [78]:
df2['release_date'] = pd.to_datetime(df2['release_date'], format='%Y/%m/%d')

In [79]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3004 entries, 0 to 3003
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   _id                 3004 non-null   object        
 1   brand               3004 non-null   object        
 2   prod_kr_name        3004 non-null   object        
 3   rescent_price       3004 non-null   int64         
 4   model_no            3004 non-null   object        
 5   release_date        3004 non-null   datetime64[ns]
 6   colors              3004 non-null   object        
 7   release_price       3004 non-null   int64         
 8   instant_buy_price   3004 non-null   int64         
 9   instant_sell_price  3004 non-null   int64         
 10  img_url             3004 non-null   object        
dtypes: datetime64[ns](1), int64(4), object(6)
memory usage: 258.3+ KB


In [80]:
df2.to_csv('kream_detail.csv', index=False)